# 1、Bilstm中文分词实验
```
一、数据集 msr_training.txt（已打好标签），测试集msr_test_gold.txt
```

In [1]:
# -*- coding:utf-8 -*-
from __future__ import print_function
import sys
print(sys.getdefaultencoding())
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

# 第一版中文分词程序：单向lstm
import re
import random
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.utils import np_utils
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

# 指定显示占用率
#import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
#set_session(tf.Session(config=config))


word_size = 128
maxlen = 128
epochs = 100
TRAIN_PERC = 0.9
batch_size = 1024 # 1024
s = []

with open('msr_train_p.txt') as inputs:
    line = inputs.readline().strip('\r\n')
    while(line):
        line = line.decode('gbk')
        s.append(line)
        line = inputs.readline().strip('\r\n')
t = []
with open('msr_test_gold_p.txt') as inputs:
    line = inputs.readline().strip('\r\n')
    while(line):
        line = line.decode('gbk')
        t.append(line)
        line = inputs.readline().strip('\r\n')
# 下面这个可以滤掉空格行        
#f = open('msr_train.txt').read().decode('gbk') # 载入数据        
#f = f.split('\r\n')
#s = []
#for line in f:
#    if line:
#        s.append(line)

ascii


Using TensorFlow backend.


In [2]:
#print(s[20000])
print("训练集句子数：",len(s))
print("测试集句子数：",len(t))

训练集句子数： 86918
测试集句子数： 3985


In [3]:
train_x = [] #生成训练样本
train_y = []
test_x = [] #生成训练样本
test_y = []

def get_xy(s):  # 把汉字与标签分开
    s = re.findall('(.)/(.)', s)
    if s:
        s = np.array(s)
        return list(s[:,0]), list(s[:,1])
    
def split_data_label(ss):
    data = [] 
    label = []
    for i in ss:  
        x = get_xy(i)
        if x:
            data.append(x[0])
            label.append(x[1])
    return data,label


# 训练集汉字与标签分开
train_x,train_y = split_data_label(s)

# 测试样本
test_x,test_y = split_data_label(t)

In [4]:
#a = '人/b  们/e  常/s  说/s  生/b  活/e  是/s  一/s  部/s  教/b  科/m  书/e '
#get_xy(a)
[print(x,end='') for x in test_x[0]]
print(test_x[0])

扬帆远东做与中国合作的先行[u'\u626c', u'\u5e06', u'\u8fdc', u'\u4e1c', u'\u505a', u'\u4e0e', u'\u4e2d', u'\u56fd', u'\u5408', u'\u4f5c', u'\u7684', u'\u5148', u'\u884c']


In [5]:
train = pd.DataFrame(index=range(len(train_x)))
train['train_x'] = train_x
train['train_y'] = train_y
train = train[train['train_x'].apply(len) <= maxlen]  # 如果大于maxlen的句子滤掉
train.index = range(len(train))
print('过滤后剩余的训练集句子数 = ',len(train))

test= pd.DataFrame(index=range(len(test_x)))
test['test_x'] = test_x
test['test_y'] = test_y
test = test[test['test_x'].apply(len) <= maxlen]  # 如果大于maxlen的句子滤掉
test.index = range(len(test))
print('过滤后剩余的测试集句子数 = ',len(test))

过滤后剩余的训练集句子数 =  85620
过滤后剩余的测试集句子数 =  3891


In [6]:
train_x = list(train['train_x'])
train_y = list(train['train_y'])

test_x = list(test['test_x'])
test_y = list(test['test_y'])

for i,line in enumerate(train_x):
    str_tmp = ''
    for char in line:
        str_tmp+=char+' '
    train_x[i] = str_tmp
    
for i,line in enumerate(train_y):
    str_tmp = ''
    for char in line:
        str_tmp+=char+' '
    train_y[i] = str_tmp

for i,line in enumerate(test_x):
    str_tmp = ''
    for char in line:
        str_tmp+=char+' '
    test_x[i] = str_tmp
    
for i,line in enumerate(test_y):
    str_tmp = ''
    for char in line:
        str_tmp+=char+' '
    test_y[i] = str_tmp

In [7]:
print(test_x[0])

# token 序列化
tokenizer_x = Tokenizer(num_words=None)
tokenizer_x.fit_on_texts(np.concatenate((train_x,test_x),axis=0))
word_index = tokenizer_x.word_index # 词_索引,字典
print("汉字个数：",len(word_index))

tokenizer_y = Tokenizer(num_words=None)
tokenizer_y.fit_on_texts(['s b m e'])
label_index = tokenizer_y.word_index

train_x = tokenizer_x.texts_to_sequences(train_x)
train_y = tokenizer_y.texts_to_sequences(train_y) 

test_x = tokenizer_x.texts_to_sequences(test_x)
test_y = tokenizer_y.texts_to_sequences(test_y)

print(label_index)

扬 帆 远 东 做 与 中 国 合 作 的 先 行 
汉字个数： 5111
{'e': 4, 's': 1, 'b': 2, 'm': 3}


In [8]:
# 上面的标签序列从1开始，改为从0开始
for i,line in enumerate(train_y):
    for j,num in enumerate(line):
        train_y[i][j] = num-1
for i,line in enumerate(test_y):
    for j,num in enumerate(line):
        test_y[i][j] = num-1

In [9]:
print(train_y[0])
print(test_y[0])

[0, 1, 3, 0, 0, 1, 3, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 2, 2, 3, 0, 1, 2, 3, 0, 0, 1, 3, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 3, 0, 0]
[1, 3, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3]


In [10]:
# 训练集填充，一个句子的字数小于maxlen，后面填充0
train_x = sequence.pad_sequences(train_x, maxlen=maxlen,padding='post')
train_y = sequence.pad_sequences(train_y, maxlen=maxlen,padding='post',value=4.)
train_y = to_categorical(train_y, num_classes=5)
train_y = train_y.reshape(-1,maxlen,5)

# 测试集要填充，否则没法用to_categoriecal函数，如果怀疑这样的准确度，可以后面再验证。
test_x = sequence.pad_sequences(test_x, maxlen=maxlen,padding='post')
test_y = sequence.pad_sequences(test_y, maxlen=maxlen,padding='post',value=4.)
test_y = to_categorical(test_y, num_classes=5)
test_y = test_y.reshape(-1,maxlen,5)

In [11]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
#print(train_x[0])
#print(train_y[0])
#print(test_x[0])
#print(test_y[0])

(85620, 128)
(85620, 128, 5)
(3891, 128)
(85620, 128, 5)


In [12]:
# 打散数据
indices = np.arange(train_x.shape[0])
np.random.shuffle(indices)
train_x = train_x[indices]
train_y = train_y[indices]

# 取部分数据
#train_x=train_x[:10000]
#train_y=train_y[:10000]


# 分开训练集与验证集
len_train = int(train_x.shape[0]*TRAIN_PERC)
len_develop = int(train_x.shape[0]-len_train)

print(len_train)
print(len_develop)

develop_x = train_x[len_train:]
develop_y = train_y[len_train:]

train_x = train_x[:len_train]
train_y = train_y[:len_train]

print(train_x.shape[0])
print(train_y.shape[0])
print(develop_x.shape[0])
print(develop_y.shape[0])

#train_y = train_y.reshape((-1,maxlen,5))
#develop_y = develop_y.reshape((-1,maxlen,5))

77058
8562
77058
77058
8562
8562


In [ ]:
#设计模型
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional
from keras.models import Model
from keras.optimizers import SGD,RMSprop,Adagrad,Adadelta,Adamax,Adam

# 函数式模型
sequence = Input(shape=(maxlen,), dtype='int32')
embedded = Embedding(len(word_index)+1, word_size, input_length=maxlen, mask_zero=True)(sequence)
blstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode='sum')(embedded)
output = TimeDistributed(Dense(5, activation='softmax'))(blstm)
model = Model(inputs=sequence, outputs=output)
#op = Adam(lr=0.0015, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#op = RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()
history = model.fit(train_x, train_y, 
                    validation_data=(develop_x,develop_y),
                    batch_size=batch_size, 
                    epochs=epochs)

In [ ]:
# 这里的测试集是有填充的，后面可以把填充的去掉试一试
score, acc = model.evaluate(test_x, test_y,
                            batch_size=batch_size,verbose=2)

print('Test score:',score)
print('Test accuracy:',acc)

# 输出部分结果
# 输出结果
#predict = model.predict_classes(test_x[:2])
#print(predict)